In [1]:
from categories_download import get_categories
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import os


In [2]:
class recipe_info:
    def __init__(self):
        self.recipe_url = None
        self.recipe_img_link = None
        self.recipe_rating = None
        self.recipe_difficulty = None
        self.recipe_time = None
        self.recipe_author = None
        self.recipe_steps = None
        self.recipe_ingredients = None
        self.recipe_accessories = None
    def f(self):
        return 'hello world'

In [3]:
def item_info_download(item):
        #download author
        author_container = item.find_all('div',class_='item-author')[0]
        item_author = author_container.a.text
#         print(item_author)
        #download rating
        item_rating_container = item.find('div',class_ = 'item-rating clearfix')
        item_rating = item_rating_container.div['data-average']
#         print(item_rating)
        #download difficulty
        item_difficulty_container = item.find('div',class_ = 'col-xs-3 difficulty-inf')
        item_difficulty = item_difficulty_container.text.strip()
#         print(item_difficulty)
        #download cooking time
        item_time_container = item.find('div',class_ = 'col-xs-3 cooking-time-inf')
        item_time = item_time_container.text.strip()
#         print(item_time)
        #download name
        item_name = item.a['title']
#         print(item_name)
        #download image link
        item_img_link = item.img['src']
#         print(item_img_link)
        #download item url
        item_url_add = item.a['href']
        item_extend_url = home_url+item_url_add
#         print(item_extend_url)
        #create data frame
        item_dic = {}
        item_dic['link'] = item_extend_url
        item_dic['img link']= item_img_link
        item_dic['rating'] = item_rating
        item_dic['difficulty'] = item_difficulty
        item_dic['cooking time'] = item_time
        item_dic['author'] = item_author

        item_info = pd.DataFrame(item_dic,index=[item_name])
#         print(item_info)
        return item_name,item_extend_url,item_dic;

In [4]:
home_url = "https://www.recipecommunity.com.au"
categories_info_file = get_categories()
categories_data = pd.read_csv(categories_info_file, index_col=0)


/******************************/
finish categories info download
/******************************/


In [5]:
categories_indices = categories_data.index
for category_index in categories_indices:
    #get item url
    item_url = categories_data['link'][category_index]
#     print(item_url)
    #download url contents and apply beautifulsoup
    item_response = get(item_url)
    item_html_soup = BeautifulSoup(item_response.text, 'html.parser')
    #find items on the first page
    items_container = item_html_soup.find_all('div', class_ = 'thumbnail result-recipe result-grid-display')
#     print(len(items_container))
    items_data = pd.DataFrame(columns=['link','img link','rating','difficulty','cooking time','author','steps','ingredients'])
    category_recipe_outfile = str(category_index)+'.csv'
    for item in items_container:
        item_name,recipe_url,item_dic = item_info_download(item)
        print("downloading "+ item_name)
        #process recipe url
        recipe_response = get(recipe_url)
        recipe_html_soup = BeautifulSoup(recipe_response.text, 'html.parser')
        #find recipe steps
        recipe_container = recipe_html_soup.find('ol', class_ = 'steps-list')
        step_list = recipe_container.find_all('li')
        #append steps to item dictionary
        steps = []
        try:
            for i in range(len(step_list)):
                steps.append(step_list[i].p.text)
        except:
            for i in range(len(step_list)):
                steps.append(" ".join(step_list[i].text.split()))
        item_dic['steps'] = [steps]
#             item_dic['Step '+str(i)] = step_list[i].p.text
        #find ingredients
        ingredients_container = recipe_html_soup.find('div', class_ = 'ingredients')
#         print(type(ingredients_container))
        ingredient_list = ingredients_container.find_all('li')
        ingredients = []
        #append ingredients to dictionary
        for i in range(len(ingredient_list)):
             ingredients.append(" ".join(ingredient_list[i].text.split()))
        item_dic['ingredients'] = [ingredients]
        item_info = pd.DataFrame(item_dic,index=[item_name])
        items_data = items_data.append(item_info)

    items_data.to_csv(category_recipe_outfile, index = True,sep=",")
    break

downloading Baby Biscuits (sugar free)
downloading Baby coconut chicken meal
downloading Apple, Pear and Cinnamon Puree (Baby Food 6+ months)
downloading Raw cacao bites
downloading Baby Rice Rusks
downloading 'Sausage' rolls
downloading Apricot Chicken Puree (Baby Food 8+ months)
downloading Fruity vegetables
downloading Red Lentil and Sweet Potato Puree (From 6 months)
downloading Baby's first meals
downloading Super Quick & Easy Banana Porridge
downloading Apple Spelt Rusks


In [6]:
items_data

,link,img link,rating,difficulty,cooking time,author,steps,ingredients
Baby Biscuits (sugar free),https://www.recipecommunity.com.au/baby-food-r...,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,4.71,easy,13min,Jessscott,"[Preheat oven to 180 degrees, Line 2 baking tr...","[60 gram rolled oats, 40 gram pitted dates, 25..."
Baby coconut chicken meal,https://www.recipecommunity.com.au/baby-food-r...,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,4.82,--,,Heidelightful,"[1. Place chicken, vegetables and coconut milk...","[120 g chicken breast, roughly chopped, 220 g ..."
"Apple, Pear and Cinnamon Puree (Baby Food 6+ months)",https://www.recipecommunity.com.au/baby-food-r...,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,4.5,easy,,Caroline Novinc,"[Place apples, pears, cinnamon and water into ...","[2 apples, peeled, cored and roughly cut, 1 pe..."
Raw cacao bites,https://www.recipecommunity.com.au/baby-food-r...,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,5,easy,2min,Louise Fulton Keats,[Place dates into mixing bowl and chop 10 sec/...,"[250 g fresh dates (such as Medjool), pitted, ..."
Baby Rice Rusks,https://www.recipecommunity.com.au/baby-food-r...,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,4.83,easy,40min,jnsnt,[Preheat oven to 180C and line a baking tray w...,"[125 g rice, 1 banana, approx 125g, peeled]"
'Sausage' rolls,https://www.recipecommunity.com.au/baby-food-r...,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,4.56,easy,,mjklimec,[1. Preheat oven to 200 degrees celcius.],"[1000 grams premium beef mince, (or mince stea..."
Apricot Chicken Puree (Baby Food 8+ months),https://www.recipecommunity.com.au/baby-food-r...,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,4.94,--,,Caroline Novinc,[Place rice into TM bowl and mill for 15 secon...,"[35g organic brown rice, 40g onion, peeled & h..."
Fruity vegetables,https://www.recipecommunity.com.au/baby-food-r...,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,4.56,easy,40min,Thermylicious,[Peel and roughly chop all iingredients],"[2 carrots, 1/2 pumpkin, 1/2 sweet potato, 1 a..."
Red Lentil and Sweet Potato Puree (From 6 months),https://www.recipecommunity.com.au/baby-food-r...,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,4.56,easy,5min,tclaven,"[Place onion, celery, zucchini and carrot in T...","[15 g extra virgin olive oil, 50 g onion, peel..."
Baby's first meals,https://www.recipecommunity.com.au/baby-food-r...,https://d2mkh7ukbp9xav.cloudfront.net/recipeim...,5,easy,,drjules,"[Sweet potato puree, Sweet potato and potato p...","[250 grams sweet potato, cubed into 2cm pieces..."


In [68]:
# item_name = 'eggs'
# recipe_url = "https://www.recipecommunity.com.au/basics-recipes/perfectly-poached-eggs/5mmwgfdy-67b84-769045-cfcd2-7x3d9v6g"
# item_dic = {}
# print("downloading "+ item_name)
# #process recipe url
# recipe_response = get(recipe_url)
# recipe_html_soup = BeautifulSoup(recipe_response.text, 'html.parser')
# #find recipe steps
# recipe_container = recipe_html_soup.find('ol', class_ = 'steps-list')
# step_list = recipe_container.find_all('li')
# #append steps to item dictionary
# steps = []
# try:
#     for i in range(len(step_list)):
#         steps.append(step_list[i].p.text)
# except:
#     for i in range(len(step_list)):
#         steps.append(" ".join(step_list[i].text.split()))
        
# item_dic['steps'] = [steps]
# #             item_dic['Step '+str(i)] = step_list[i].p.text
# #find ingredients
# ingredients_container = recipe_html_soup.find('div', class_ = 'ingredients')
# #         print(type(ingredients_container))
# ingredient_list = ingredients_container.find_all('li')
# ingredients = []
# #append ingredients to dictionary
# for i in range(len(ingredient_list)):
#      ingredients.append(" ".join(ingredient_list[i].text.split()))
# item_dic['ingredients'] = [ingredients]
# item_info = pd.DataFrame(item_dic,index=[item_name])
# items_data = items_data.append(item_info)
# print(items_data)

downloading eggs
     author cooking time difficulty img link  \
eggs    NaN          NaN        NaN      NaN   
eggs    NaN          NaN        NaN      NaN   

                                            ingredients link rating  \
eggs  [59 grams eggs, Or XL size, 1.8 litres water, ...  NaN    NaN   
eggs  [59 grams eggs, Or XL size, 1.8 litres water, ...  NaN    NaN   

                                                  steps  
eggs  1. Place steamer basket and 1.8ltrs water into...  
eggs  [1. Place steamer basket and 1.8ltrs water int...  


In [69]:
# item_dic

{'steps': [['1. Place steamer basket and 1.8ltrs water into the thermomix bowl. Heat 8 min 70deg speed 3. 2. Place eggs (I used 59gm xl eggs from the fridge) carefully into the steamer basket. 3. Cook 11 minutes 70deg speed 3. 4. Carefully crack the egg on the pointy end and slight peel back some shell. The egg will appear poached and should slide out the shell. Scoop out if any remaining egg white from the shell with a spoon. Yum!',
   '1. Place steamer basket and 2ltrs water into the thermomix bowl. Heat 7 min 70deg speed 3.5. 2. Place eggs (I used 59gm xl eggs from the fridge) carefully into the steamer basket. 3. Cook 12.5 minutes 70deg speed 3.5. 4. Carefully crack the egg on the pointy end and slight peel back some shell. The egg will appear poached and should slide out the shell. Scoop out if any remaining egg white from the shell with a spoon. Yum!',
   "Sourced method from Alyce Alexandra This cooking method is known as Sous Vide (slow cooked). I've cut the egg in the photo to

In [73]:
recipe_url = "https://www.recipecommunity.com.au/baby-food-recipes/baby-coconut-chicken-meal/4wyfnaab-f9c28-481590-cfcd2-xs7vp6mu"
item_name = "a"
item_dic = {}
print("downloading "+ item_name)
#process recipe url
recipe_response = get(recipe_url)
recipe_html_soup = BeautifulSoup(recipe_response.text, 'html.parser')
#find recipe steps
recipe_container = recipe_html_soup.find('ol', class_ = 'steps-list')
step_list = recipe_container.find_all('li')
#append steps to item dictionary
steps = []
try:
    for i in range(len(step_list)):
        steps.append(step_list[i].p.text)
except:
    for i in range(len(step_list)):
        steps.append(" ".join(step_list[i].text.split()))
item_dic['steps'] = [steps]
#             item_dic['Step '+str(i)] = step_list[i].p.text
#find ingredients
ingredients_container = recipe_html_soup.find('div', class_ = 'ingredients')
#         print(type(ingredients_container))
ingredient_list = ingredients_container.find_all('li')
ingredients = []
#append ingredients to dictionary
for i in range(len(ingredient_list)):
     ingredients.append(" ".join(ingredient_list[i].text.split()))
item_dic['ingredients'] = [ingredients]
item_info = pd.DataFrame(item_dic,index=[item_name])
items_data = items_data.append(item_info)
print(item_dic)

downloading a
{'steps': [['1. Place chicken, vegetables and coconut milk into TM bowl.\xa0']], 'ingredients': [['120 g chicken breast, roughly chopped', '220 g vegetables/fruits, eg. pumpkin, sweet potato, carrot, broccoli, or other age-appropriate veges', '85 g coconut milk/cream']]}


In [106]:
accessories_container = recipe_html_soup.find('div', class_ = 'accessories-list')

In [105]:
tools_list = accessories_container.find_all("meta")

In [111]:
recipe_html_soup.find(itemprop="tool").get("content")

'Spatula'

In [113]:
tools_list[0].get("content")

'Spatula'

'Spatula'